In [9]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [37]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database='postgres')

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
f = 1.0                                            # desired frac
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT content, type_id FROM fakenews.article", connection).sample(frac=f, random_state=seed)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df

,content,type_id
7576,from reason magazine which really ought to loo...,2
10509,christians are being annihilated — wiped from ...,3
4253,i wrote about the nfl ratings about a month ag...,1
5150,the pm and the opposition leader have both exp...,1
506,headline bitcoin blockchain searches exceed tr...,0
...,...,...
905,weekly crude oil gold silver coffee and sugar ...,0
5192,NUM k shares share the physician white coat c...,1
12172,natural news brevard county sheriff wayne ivey...,4
235,headline bitcoin blockchain searches exceed tr...,0


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

x = v.fit_transform(df['content'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, df["type_id"], test_size=0.2, random_state=0)

In [48]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(11, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [49]:
model.fit(X_train.toarray(), y_train.values, epochs=3)

MemoryError: 

In [80]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test.toarray(),y_test.values)
print(val_loss, val_acc)

20/20 [==============================] - 0s 9ms/sample - loss: 0.6735 - accuracy: 1.0000
0.6735154390335083 1.0
